In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [2]:
df = pd.read_csv('Bib_skoky\David-50-pokus01\Xsens_DOT_3_D422CD006379_20250409_104913.csv', skiprows=7, sep=",")
df.head(10)

,PacketCounter,SampleTimeFine,Euler_X,Euler_Y,Euler_Z,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z
0,0,1607860018,-173.586258,-88.626701,107.056412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1607876685,-173.670959,-88.621437,107.157219,9.773009,0.009794,-0.249013,1.399881,0.141555,0.222991
2,2,1607893352,-173.730133,-88.616058,107.225388,9.758099,0.029466,-0.259871,1.741350,0.131963,0.253201
3,3,1607910019,-173.921432,-88.613228,107.411781,9.798561,0.021218,-0.240829,0.382494,0.273435,0.075461
4,4,1607926686,-174.119919,-88.608162,107.609444,9.813557,-0.038344,-0.320160,0.771743,0.166937,0.065421
5,5,1607943353,-174.240021,-88.605064,107.735283,9.795274,0.019422,-0.259917,0.892340,0.278967,0.164325
6,6,1607960020,-174.394745,-88.601707,107.890450,9.759882,-0.008067,-0.326875,0.812108,0.246554,0.118209
7,7,1607976687,-174.422974,-88.593895,107.932983,9.751965,0.016170,-0.310375,1.639258,0.005577,0.307051
8,8,1607993354,-174.591187,-88.585426,108.110641,9.773182,0.029907,-0.266095,0.917902,-0.063391,0.121359
9,9,1608010021,-174.844254,-88.575912,108.359406,9.789434,0.000883,-0.244162,0.738958,-0.140037,0.002888


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PacketCounter   672 non-null    int64  
 1   SampleTimeFine  672 non-null    int64  
 2   Euler_X         672 non-null    float64
 3   Euler_Y         672 non-null    float64
 4   Euler_Z         672 non-null    float64
 5   Acc_X           672 non-null    float64
 6   Acc_Y           672 non-null    float64
 7   Acc_Z           672 non-null    float64
 8   Gyr_X           672 non-null    float64
 9   Gyr_Y           672 non-null    float64
 10  Gyr_Z           672 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 57.9 KB


In [4]:
df.describe()

,PacketCounter,SampleTimeFine,Euler_X,Euler_Y,Euler_Z,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z
count,672.000000,6.720000e+02,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000
mean,335.500000,1.613452e+09,-74.479460,-84.888384,90.523244,9.670396,0.016180,-0.848848,1.127698,0.526937,0.305965
std,194.133974,3.235631e+06,132.340051,8.740592,40.243545,7.700914,3.781388,8.596066,72.337476,74.666431,15.780066
min,0.000000,1.607860e+09,-179.987091,-89.170311,-37.682327,-33.603298,-43.923359,-121.898544,-917.986389,-603.439880,-80.111992
25%,167.750000,1.610656e+09,-171.971142,-88.618673,65.450832,9.751988,-0.247142,-0.424608,0.011444,-1.633398,-0.416706
50%,335.500000,1.613452e+09,-135.799606,-87.930676,104.310226,9.779851,-0.003289,-0.293431,1.112859,0.254494,0.274274
75%,503.250000,1.616248e+09,-46.889475,-87.133997,114.110794,9.794640,0.037567,-0.211629,1.625481,1.236642,0.461970
max,671.000000,1.619044e+09,179.997772,-36.454224,149.749069,91.325966,52.167519,90.000832,405.980408,593.437256,125.089149


In [5]:
# I zero the clock at the first row, this makes all subsequent rows relative to test start
df['real_ticks'] = df['SampleTimeFine'] - df['SampleTimeFine'].iloc[0] 

# Converting the ticks to seconds and milliseconds
df['time_s'] = df['real_ticks'] * 1e-6

# Convert seconds to milliseconds
df['time_ms'] = df['time_s'] * 1000
#  show the actual intervals between samples
df['delta_ms'] = df['time_ms'].diff()

In [7]:
df[['time_ms', 'time_s', 'delta_ms']].describe()

,time_ms,time_s,delta_ms
count,672.000000,672.000000,6.710000e+02
mean,5591.778500,5.591778,1.666700e+01
std,3235.630951,3.235631,5.193370e-13
min,0.000000,0.000000,1.666700e+01
25%,2795.889250,2.795889,1.666700e+01
50%,5591.778500,5.591778,1.666700e+01
75%,8387.667750,8.387668,1.666700e+01
max,11183.557000,11.183557,1.666700e+01


In [17]:
print('Z axis acceleration statistics of left foot IMU vs Time in Seconds, Milliseconds and Delta in milliseconds:\n')
df[['time_ms', 'time_s', 'delta_ms', 'Acc_Z']].describe()

Z axis acceleration statistics of left foot IMU vs Time in Seconds, Milliseconds and Delta in milliseconds:



,time_ms,time_s,delta_ms,Acc_Z
count,672.000000,672.000000,6.710000e+02,672.000000
mean,5591.778500,5.591778,1.666700e+01,-0.848848
std,3235.630951,3.235631,5.193370e-13,8.596066
min,0.000000,0.000000,1.666700e+01,-121.898544
25%,2795.889250,2.795889,1.666700e+01,-0.424608
50%,5591.778500,5.591778,1.666700e+01,-0.293431
75%,8387.667750,8.387668,1.666700e+01,-0.211629
max,11183.557000,11.183557,1.666700e+01,90.000832


In [6]:
acc_magnitude = np.sqrt(df["Acc_X"]**2 + df["Acc_Y"]**2 + df["Acc_Z"]**2)

acc_magnitude.describe()

count    672.000000
mean      11.456074
std       10.508849
min        0.000000
25%        9.771140
50%        9.788025
75%        9.808715
max      127.857470
dtype: float64